In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('data/dhs_clusters.csv')
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))
print(df.info())

| country   | survey_start_year   | year   | lat      | lon     | GID_1   | GID_2    | households   | rural   | iwi     |
|:----------|:--------------------|:-------|:---------|:--------|:--------|:---------|:-------------|:--------|:--------|
| angola    | 2006                | 2006   | -12.1693 | 13.8593 | AGO.2   | AGO.2.9  | 20           | 0       | 68.0963 |
| angola    | 2006                | 2006   | -12.3617 | 14.7782 | AGO.2   | AGO.2.2  | 19           | 1       | 10.6091 |
| angola    | 2006                | 2006   | -10.8275 | 14.3421 | AGO.7   | AGO.7.1  | 25           | 1       | 29.9739 |
| angola    | 2006                | 2006   | -9.99131 | 14.8983 | AGO.7   | AGO.7.5  | 1            | 1       | 7.70413 |
| angola    | 2006                | 2006   | -11.5189 | 14.7504 | AGO.7   | AGO.7.10 | 24           | 1       | 13.3718 |
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57195 entries, 0 to 57194
Data columns (total 10 columns):
 #   Column             Non-Null Coun

In [ ]:
min_year = df['year'].min()
max_year = df['year'].max()
all_years = range(min_year, max_year + 1)
unique_countries = df['country'].unique()
multi_index = pd.MultiIndex.from_product([unique_countries, all_years], names=['country', 'year'])
grouped = df.groupby(['country', 'year'])
summary_df = grouped.agg(
    total_count=('year', 'size'),  
    rural_count=('rural', 'sum')  
).reset_index()
summary_df = summary_df.set_index(['country', 'year'])
summary_df = summary_df.reindex(multi_index, fill_value=0)
summary_df['urban_count'] = summary_df['total_count'] - summary_df['rural_count']
summary_df = summary_df.reset_index()
print(summary_df.to_markdown(index=False, numalign="left", stralign="left"))

| country                      | year   | total_count   | rural_count   | urban_count   |
|:-----------------------------|:-------|:--------------|:--------------|:--------------|
| angola                       | 1991   | 0             | 0             | 0             |
| angola                       | 1992   | 0             | 0             | 0             |
| angola                       | 1993   | 0             | 0             | 0             |
| angola                       | 1994   | 0             | 0             | 0             |
| angola                       | 1995   | 0             | 0             | 0             |
| angola                       | 1996   | 0             | 0             | 0             |
| angola                       | 1997   | 0             | 0             | 0             |
| angola                       | 1998   | 0             | 0             | 0             |
| angola                       | 1999   | 0             | 0             | 0             |
| angola  

In [ ]:
total_years_in_range = max_year - min_year + 1

country_summary = summary_df.groupby('country').agg(
    years_with_data=('total_count', lambda x: (x > 0).sum()), 
    total_data_points=('total_count', 'sum'),
    total_rural_count=('rural_count', 'sum'),
    total_urban_count=('urban_count', 'sum')
).reset_index() 

country_summary['years_with_zero_data'] = total_years_in_range - country_summary['years_with_data']
country_summary['max_year_range'] = f"{min_year}-{max_year}"
country_summary = country_summary[[
    'country',
    'max_year_range',
    'years_with_data',
    'years_with_zero_data',
    'total_data_points',
    'total_rural_count',
    'total_urban_count'
]]

print(country_summary.to_markdown(index=False, numalign="left", stralign="left"))

| country                      | max_year_range   | years_with_data   | years_with_zero_data   | total_data_points   | total_rural_count   | total_urban_count   |
|:-----------------------------|:-----------------|:------------------|:-----------------------|:--------------------|:--------------------|:--------------------|
| angola                       | 1991-2019        | 6                 | 23                     | 969                 | 483                 | 486                 |
| benin                        | 1991-2019        | 6                 | 23                     | 1710                | 954                 | 756                 |
| burkina_faso                 | 1991-2019        | 9                 | 20                     | 1740                | 1198                | 542                 |
| burundi                      | 1991-2019        | 5                 | 24                     | 1128                | 911                 | 217                 |
| cameroon            

In [ ]:
min_iwi = df['iwi'].min()
max_iwi = df['iwi'].max()
print(f"Min IWI: {min_iwi}")
print(f"Max IWI: {max_iwi}")
bin_size = 10
bin_edges = np.arange(np.floor(min_iwi / bin_size) * bin_size, np.ceil(max_iwi / bin_size) * bin_size + bin_size, bin_size)
if max_iwi == bin_edges[-1]:
     bin_edges[-1] += 0.1 

print(f"Using bins: {bin_edges}")
bin_labels = [f"{int(bin_edges[i])}-{int(bin_edges[i+1])}" for i in range(len(bin_edges)-1)]
df['iwi_bin'] = pd.cut(df['iwi'], bins=bin_edges, labels=bin_labels, right=False, include_lowest=True)
print("\nIWI Bin distribution check:")
print(df['iwi_bin'].value_counts().sort_index().to_markdown(numalign="left", stralign="left"))
iwi_counts_per_country = df.groupby(['country', 'iwi_bin'], observed=False).size().unstack(fill_value=0)
iwi_counts_per_country.columns = [f"iwi_bin_{label}_count" for label in iwi_counts_per_country.columns]
print("\nIWI Bin counts per country:")
print(iwi_counts_per_country.to_markdown(numalign="left", stralign="left"))
min_year = df['year'].min()
max_year = df['year'].max()
all_years = range(min_year, max_year + 1)
unique_countries = df['country'].unique()
multi_index = pd.MultiIndex.from_product([unique_countries, all_years], names=['country', 'year'])
grouped = df.groupby(['country', 'year'])
temp_summary_df = grouped.agg(
    total_count=('year', 'size'),
    rural_count=('rural', 'sum')
).reset_index()
temp_summary_df = temp_summary_df.set_index(['country', 'year'])
temp_summary_df = temp_summary_df.reindex(multi_index, fill_value=0)
temp_summary_df['urban_count'] = temp_summary_df['total_count'] - temp_summary_df['rural_count']
temp_summary_df = temp_summary_df.reset_index()
country_base_summary = temp_summary_df.groupby('country').agg(
    years_with_data=('total_count', lambda x: (x > 0).sum()),
    total_data_points=('total_count', 'sum'),
    total_rural_count=('rural_count', 'sum'),
    total_urban_count=('urban_count', 'sum')
)
final_country_summary = country_base_summary.join(iwi_counts_per_country)
total_years_in_range = max_year - min_year + 1
final_country_summary['years_with_zero_data'] = total_years_in_range - final_country_summary['years_with_data']
final_country_summary['max_year_range'] = f"{min_year}-{max_year}"
final_country_summary = final_country_summary.reset_index()
base_cols = [
    'country', 'max_year_range', 'years_with_data', 'years_with_zero_data',
    'total_data_points', 'total_rural_count', 'total_urban_count'
]
iwi_cols = sorted([col for col in final_country_summary.columns if col.startswith('iwi_bin_')])
final_country_summary = final_country_summary[base_cols + iwi_cols]
print("\nFinal Country Summary with IWI Bins:")
print(final_country_summary.to_markdown(index=False, numalign="left", stralign="left"))

Min IWI: -1.83416
Max IWI: 86.065248
Using bins: [-10.   0.  10.  20.  30.  40.  50.  60.  70.  80.  90.]

IWI Bin distribution check:
| iwi_bin   | count   |
|:----------|:--------|
| -10-0     | 27      |
| 0-10      | 4622    |
| 10-20     | 16476   |
| 20-30     | 10292   |
| 30-40     | 6759    |
| 40-50     | 6339    |
| 50-60     | 5702    |
| 60-70     | 4762    |
| 70-80     | 2148    |
| 80-90     | 68      |

IWI Bin counts per country:
| country                      | iwi_bin_-10-0_count   | iwi_bin_0-10_count   | iwi_bin_10-20_count   | iwi_bin_20-30_count   | iwi_bin_30-40_count   | iwi_bin_40-50_count   | iwi_bin_50-60_count   | iwi_bin_60-70_count   | iwi_bin_70-80_count   | iwi_bin_80-90_count   |
|:-----------------------------|:----------------------|:---------------------|:----------------------|:----------------------|:----------------------|:----------------------|:----------------------|:----------------------|:----------------------|:----------------------|
| an

In [ ]:
iwi_count_cols = sorted([col for col in final_country_summary.columns if col.startswith('iwi_bin_')])
country_iwi_counts = final_country_summary[['country'] + iwi_count_cols].set_index('country')

country_iwi_counts['iwi_count_variance'] = country_iwi_counts[iwi_count_cols].var(axis=1, ddof=1)

variance_results = country_iwi_counts[['iwi_count_variance']].reset_index()

min_variance_row = variance_results.loc[variance_results['iwi_count_variance'].idxmin()]
min_variance_country = min_variance_row['country']
min_variance_value = min_variance_row['iwi_count_variance']

print(f"Country with the lowest variance in IWI bin counts: {min_variance_country} (Variance: {min_variance_value:.2f})\n")

print("Variance of IWI bin counts for all countries:")
print(variance_results.to_markdown(index=False, numalign="left", stralign="left"))

Country with the lowest variance in IWI bin counts: comoros (Variance: 796.84)

Variance of IWI bin counts for all countries:
| country                      | iwi_count_variance   |
|:-----------------------------|:---------------------|
| angola                       | 5466.54              |
| benin                        | 39765.1              |
| burkina_faso                 | 53598.4              |
| burundi                      | 51077.7              |
| cameroon                     | 21189.6              |
| central_african_republic     | 1676.5               |
| chad                         | 10849.2              |
| comoros                      | 796.844              |
| democratic_republic_of_congo | 13151.3              |
| egypt                        | 950080               |
| eswatini                     | 1061.11              |
| ethiopia                     | 134817               |
| gabon                        | 1055.96              |
| ghana                        | 4

In [ ]:
print("--- Calculating Metrics ---")
year_stats = temp_summary_df[temp_summary_df['total_count'] > 0].groupby('country')['year'].agg(['min', 'max'])
if not year_stats.empty:
    year_stats['year_span'] = year_stats['max'] - year_stats['min'] + 1
else:
    year_stats['year_span'] = 0

temporal_variance = temp_summary_df.groupby('country')['total_count'].var(ddof=0).rename('temporal_data_variance')

country_scores = final_country_summary[['country', 'total_data_points', 'years_with_data', 'total_rural_count', 'total_urban_count']].set_index('country')

max_counts = np.maximum(country_scores['total_rural_count'], country_scores['total_urban_count'])
min_counts = np.minimum(country_scores['total_rural_count'], country_scores['total_urban_count'])

country_scores['rural_urban_balance'] = np.where(
    max_counts > 0,
    min_counts / max_counts,
    np.where(country_scores['total_data_points'] == 0, 1, 0) 
)

country_scores = country_scores.join(year_stats[['year_span']], how='left') 
country_scores = country_scores.join(variance_results.set_index('country'), how='left')
country_scores = country_scores.join(temporal_variance, how='left')

country_scores['temporal_data_variance'] = country_scores['temporal_data_variance'].fillna(0)
country_scores['year_span'] = country_scores['year_span'].fillna(0)
country_scores['iwi_count_variance'] = country_scores['iwi_count_variance'].fillna(country_scores['iwi_count_variance'].max()) 

country_scores = country_scores[[
    'total_data_points',
    'years_with_data',
    'year_span',
    'iwi_count_variance',
    'temporal_data_variance',
    'rural_urban_balance'
]].copy() 

print("\n--- Raw Scores ---")
print(country_scores.reset_index().to_markdown(index=False, numalign="left", stralign="left"))

print("\n--- Normalizing Scores ---")
metrics_to_normalize = country_scores.columns
normalized_scores = country_scores.copy()

if len(country_scores) > 1:
    print("Normalizing across multiple countries...")
    for metric in metrics_to_normalize:
        min_val = country_scores[metric].min()
        max_val = country_scores[metric].max()
        if max_val - min_val > 0:
            normalized_scores[metric] = (country_scores[metric] - min_val) / (max_val - min_val)
        else:
            normalized_scores[metric] = 0.5

    normalized_scores['iwi_count_variance'] = 1.0 - normalized_scores['iwi_count_variance']
    normalized_scores['temporal_data_variance'] = 1.0 - normalized_scores['temporal_data_variance']
    print("Inverted variance scores.")

else:
    print("Assigning default 0.5 normalized score: Only one country in the data.")
    for metric in metrics_to_normalize:
         normalized_scores[metric] = 0.5

normalized_scores.columns = [f"{col}_norm" for col in normalized_scores.columns]
print("\n--- Normalized Scores ---")
print(normalized_scores.reset_index().to_markdown(index=False, numalign="left", stralign="left"))

weights = {
    'total_data_points_norm': 0.30,
    'iwi_count_variance_norm': 0.25, 
    'years_with_data_norm': 0.15,
    'temporal_data_variance_norm': 0.15, 
    'rural_urban_balance_norm': 0.10,
    'year_span_norm': 0.05,
}
if not np.isclose(sum(weights.values()), 1.0):
    print(f"Warning: Weights do not sum to 1.0 (Sum = {sum(weights.values())})")

print("\n--- Computing Final Weighted Score ---")
final_scores = normalized_scores.copy()
final_scores['final_suitability_score'] = 0.0
for metric_norm, weight in weights.items():
    if metric_norm in final_scores.columns:
        final_scores['final_suitability_score'] += final_scores[metric_norm] * weight
    else:
        print(f"Warning: Metric {metric_norm} not found in normalized scores.")

print("\n--- Final Scores ---")
final_scores_sorted = final_scores.reset_index()[['country', 'final_suitability_score']].sort_values('final_suitability_score', ascending=False)
print(final_scores_sorted.to_markdown(index=False, numalign="left", stralign="left"))

if not final_scores_sorted.empty:
    best_country = final_scores_sorted.iloc[0]['country']
    best_score = final_scores_sorted.iloc[0]['final_suitability_score']
    print(f"\nRecommended country based on this scoring: {best_country} (Score: {best_score:.3f})")
else:
    print("\nNo countries found to recommend.")

--- Calculating Metrics ---

--- Raw Scores ---
| country                      | total_data_points   | years_with_data   | year_span   | iwi_count_variance   | temporal_data_variance   | rural_urban_balance   |
|:-----------------------------|:--------------------|:------------------|:------------|:---------------------|:-------------------------|:----------------------|
| angola                       | 969                 | 6                 | 11          | 5466.54              | 7620.1                   | 0.993827              |
| benin                        | 1710                | 6                 | 23          | 39765.1              | 20221.7                  | 0.792453              |
| burkina_faso                 | 1740                | 9                 | 27          | 53598.4              | 16334                    | 0.452421              |
| burundi                      | 1128                | 5                 | 8           | 51077.7              | 9255.06                  

In [ ]:
k = 10 

print(f"\n--- Top {k} Recommended Countries ---")
final_scores_sorted = final_scores.reset_index()[['country', 'final_suitability_score']].sort_values('final_suitability_score', ascending=False)

top_k_countries = final_scores_sorted.head(k)

if not top_k_countries.empty:
    print(top_k_countries.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("\nNo countries found to rank.")


--- Top 10 Recommended Countries ---
| country      | final_suitability_score   |
|:-------------|:--------------------------|
| senegal      | 0.722182                  |
| ghana        | 0.693923                  |
| uganda       | 0.636364                  |
| cameroon     | 0.627937                  |
| burkina_faso | 0.621748                  |
| benin        | 0.618731                  |
| tanzania     | 0.617016                  |
| mali         | 0.616299                  |
| angola       | 0.610758                  |
| nigeria      | 0.610611                  |


In [ ]:
yearly_counts = temp_summary_df[['country', 'year', 'total_count']]

yearly_pivot_df = yearly_counts.pivot_table(
    index='country',
    columns='year',
    values='total_count',
    fill_value=0 
)

yearly_pivot_df = yearly_pivot_df.astype(int)

print("Data count per country per year:")
print(yearly_pivot_df.reset_index().to_markdown(index=False, numalign="left", stralign="left"))

Data count per country per year:
| country                      | 1991   | 1992   | 1993   | 1994   | 1995   | 1996   | 1997   | 1998   | 1999   | 2000   | 2001   | 2002   | 2003   | 2004   | 2005   | 2006   | 2007   | 2008   | 2009   | 2010   | 2011   | 2012   | 2013   | 2014   | 2015   | 2016   | 2017   | 2018   | 2019   |
|:-----------------------------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|
| angola                       | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 62     | 53     | 0      | 0      | 2      | 228    | 0      | 0      | 0      | 316    | 308    | 0      | 0      | 0      |
| benin                        | 0      | 0      | 0      | 0      | 0      | 190    